# Diving into French Presidential Discourses
## Eric Zemmour (EZ)

In [ ]:
!pip install minet
!pip install pdfminer.six
!pip install unidecode
!python -m spacy download fr_core_news_sm

In [1]:
import pandas as pd
from pdfminer.high_level import extract_text
from gensim.test.utils import datapath
from gensim import utils
import spacy
from collections import Counter
import unidecode
import gensim.models

In [2]:
## create a .csv of EM's Twitter activity since 27 Jan 2022
!minet tw scrape tweets "(from:ZemmourEric) until:2022-03-27 since:2022-01-27" > tweets_EZ.csv

## convert the .csv file in a data frame using pandas
df_tw_EZ = pd.read_csv("./tweets_EZ.csv")

## create a list of tweets selecting the 'text' column of the data frame
list_tw_EZ = df_tw_EZ['text'].values.tolist()
len(list_tw_EZ)

Searching for "(from:ZemmourEric) until:2022-03-27 since:2022-01-27"


1348

In [3]:
## retrieve EZ's affiliates Twitter activity
!minet tw scrape tweets "(from:Samuel_Lafont OR from:GilbertCollard OR from:jerome_riviere OR from:MarionMarechal OR from:G_Peltier OR from:NicolasBay_ OR from:DenisCieslik OR from:stanislasrig OR from:AntoineDiers OR from:de_beaujeu OR from:Stephane_Ravier OR from:MaxettePirbakas OR from:LaurenceTrochu) until:2022-03-27 since:2022-01-27" > tw_EZ_aff_all.csv

Searching for "(from:Samuel_Lafont OR from:GilbertCollard OR from:jerome_riviere OR from:MarionMarechal OR from:G_Peltier OR from:NicolasBay_ OR from:DenisCieslik OR from:stanislasrig OR from:AntoineDiers OR from:de_beaujeu OR from:Stephane_Ravier OR from:MaxettePirbakas OR from:LaurenceTrochu) until:2022-03-27 since:2022-01-27"


In [4]:
## convert EZ's affiliates' tweets in a list
df_tw_EZ_aff_all = pd.read_csv("tw_EZ_aff_all.csv")
list_tw_EZ_aff_all = df_tw_EZ_aff_all['text'].values.tolist()
print(list_tw_EZ_aff_all[0])
print(len(list_tw_EZ_aff_all))

À VOUS ! 

#ZemmourTrocadero « Samuel_Lafont: 🔴 Tweetez #ZemmourTrocadero maintenant pour soutenir Eric Zemmour ! — https://twitter.com/Samuel_Lafont/status/1507657549591597064 »
9868


In [5]:
## merge EZ and his affiliates' lists
list_tw_EZ_all = list_tw_EZ + list_tw_EZ_aff_all

In [6]:
## i retrieve a string from the pdf of EZ's manifesto using extract_text of the pdfminer package
## the cleaning process is specific for this manifesto and it depends on the output of extract_text
manif_EZ = extract_text('/Users/simonemariaparazzoli/Documents/Università/Sciences Po/Diving into public digital spaces/research/manifesto_zemmour.pdf')
manif_clean_EZ = manif_EZ.replace('-\n','')
manif_clean_EZ = manif_clean_EZ.replace('\n\n','---')
manif_clean_EZ = manif_clean_EZ.replace('\n','')
manif_clean_EZ = manif_clean_EZ.replace('\xa0','')
manif_clean_EZ = manif_clean_EZ.replace('\x0c','')
manif_clean_EZ = manif_clean_EZ.replace('. .','')
manif_clean_EZ = manif_clean_EZ.replace('  ','')
manif_clean_EZ = manif_clean_EZ.replace('. ','---')
manif_clean_EZ = manif_clean_EZ.replace('------','---')
#print(repr(manif_clean_EZ))

## convert the string of the manifesto into a list
list_manif_EZ = manif_clean_EZ.split("---")
list_manif_EZ = [s for s in list_manif_EZ if len(s)>30]
len(list_manif_EZ)
#print(list_manif_EZ)

669

In [11]:
## merge the two lists of tweets and of the manifesto 
list_EZ = list_tw_EZ_all + list_manif_EZ
len(list_EZ)

## load a spacy model to retrieve stop words
nlp = spacy.load("fr_core_news_sm")
stop_words_fr = nlp.Defaults.stop_words

## clean the list of tweets and manifesto to get rid of useless words and make the list content homogeneous
list_EZ_clean = []
for i in list_EZ:
    doc = nlp(i)
    tokens = [unidecode.unidecode(token.text).lower() for token in doc 
              if (token.text not in stop_words_fr and
                  len(token.text)>2 and
                  token.like_url == False )]
    tokens_joined = ' '.join(tokens)
    list_EZ_clean.append(tokens_joined)
    
## test the output of the cleaning process
print(list_EZ[205])
print("---")
print(len(list_EZ_clean))

Beaucoup des électeurs du RN m’ont rejoint. Marine Le Pen les a méprisés, mais elle n’a pas vu qu’en réalité, c’était elle qui les avait déçus. Ces électeurs du RN, comme je les aime et comme je les comprends : ils sont à l’avant-garde du combat pour la France.
#ZemmourMetz
---
11885


In [54]:
## prepare the corpus as a class
class MyCorpus_EZ:

    def __iter__(self):
        for i in list_EZ_clean:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(i,min_len=3)
            
## train the word embeddings model_EZ
sentences = MyCorpus_EZ()
model_EZ = gensim.models.Word2Vec(sentences=sentences, min_count=10, vector_size=300, epochs=100)

In [55]:
## transform the corpus list (that is made of tweets and sentences from the manifesto)
## in a list containing all the words of the corpus as elements of the list
words_EZ = []

for i in list_EZ_clean:
    i_split = i.split(' ') #transform the i document into a list (split at blank space)
    words_EZ.extend(i_split)

## clean the list of tokens
words_EZ_clean = [x for x in words_EZ 
                   if x not in stop_words_fr
                   if x != "\n\n"
                   if len(x)>1]

## find the 30 most common words using Counter
words_freq_EZ = Counter(words_EZ_clean)
common_words_EZ = words_freq_EZ.most_common(30)
print(common_words_EZ)

[('zemmour', 4740), ('eric', 4401), ('france', 1378), ('francais', 1110), ('partagez', 1076), ('macron', 1064), ('@zemmoureric', 939), ('soutenez', 818), ('zemmourpresident', 787), ('soutenir', 771), ('pecresse', 565), ('meeting', 517), ('faire', 465), ('remigration', 445), ('video', 443), ('contre', 429), ('direct', 426), ('grand', 392), ('elysee2022', 379), ('zemmourlille', 375), ('programme', 373), ('avez', 352), ('campagne', 349), ('soir', 345), ('pouvoirdachat', 337), ('faceababa', 326), ('pays', 320), ('000', 319), ('@vpecresse', 316), ('zemmourtoulon', 310)]


In [57]:
result = model_EZ.wv.most_similar(positive=['france'], topn=20)
print(result)

[('patrie', 0.3734579086303711), ('nation', 0.35627859830856323), ('destin', 0.3193292021751404), ('aimer', 0.30802884697914124), ('hommage', 0.2998584508895874), ('pays', 0.29895031452178955), ('redevienne', 0.2977248728275299), ('joie', 0.2958369553089142), ('pleinement', 0.2859092652797699), ('compatriotes', 0.2831250727176666), ('prospere', 0.2811393737792969), ('identite', 0.27341964840888977), ('prosperite', 0.27276137471199036), ('espoir', 0.27004003524780273), ('bats', 0.2642383277416229), ('ensemble', 0.26401323080062866), ('reconquerir', 0.2622750401496887), ('accueillir', 0.26144224405288696), ('transmettre', 0.2592829465866089), ('redevenir', 0.25566571950912476)]


In [58]:
result = model_EZ.wv.most_similar(positive=['etat'], topn=20)
print(result)

[('asile', 0.3433983027935028), ('banlieues', 0.3129144310951233), ('remettre', 0.3065248131752014), ('financement', 0.30632713437080383), ('developpement', 0.3027193248271942), ('domicile', 0.30169960856437683), ('corps', 0.29760676622390747), ('processus', 0.2968727946281433), ('depense', 0.28652796149253845), ('soignants', 0.28238868713378906), ('medicaux', 0.27827537059783936), ('aide', 0.2769870162010193), ('remboursement', 0.2758648097515106), ('fonction', 0.2708013355731964), ('revaloriser', 0.26988375186920166), ('commun', 0.268997460603714), ('administration', 0.26853010058403015), ('deserts', 0.2642192244529724), ('afrique', 0.2636655271053314), ('finances', 0.25990045070648193)]


In [59]:
result = model_EZ.wv.most_similar(positive=['souverainete'], topn=20)
print(result)

[('peuples', 0.41333869099617004), ('integrite', 0.4109932482242584), ('agricole', 0.4095165729522705), ('garantir', 0.40734368562698364), ('prosperite', 0.39261946082115173), ('independance', 0.35949113965034485), ('energetique', 0.3582710027694702), ('militaire', 0.35098597407341003), ('diplomatie', 0.33372020721435547), ('identite', 0.3290654420852661), ('recherche', 0.32527896761894226), ('gaulle', 0.32043129205703735), ('frontieres', 0.31784510612487793), ('echange', 0.3110473155975342), ('production', 0.30643725395202637), ('renforcer', 0.30587416887283325), ('nations', 0.3052631914615631), ('europe', 0.3046526312828064), ('commission', 0.2948145568370819), ('logique', 0.29455330967903137)]


In [60]:
result = model_EZ.wv.most_similar(positive=['president'], topn=20)
print(result)

[('presidence', 0.4050086438655853), ('detriment', 0.2778016924858093), ('penser', 0.2698923945426941), ('elu', 0.2685350477695465), ('mission', 0.26786869764328003), ('compatible', 0.2581610381603241), ('ferme', 0.24885018169879913), ('sort', 0.24358536303043365), ('maintenir', 0.24018198251724243), ('vendredi', 0.2379762828350067), ('devoir', 0.23764267563819885), ('diplomatique', 0.23643821477890015), ('credible', 0.23584601283073425), ('indigne', 0.23190726339817047), ('ancien', 0.22503332793712616), ('interets', 0.2249889373779297), ('engage', 0.22413717210292816), ('peut', 0.22388955950737), ('ruthelkrief', 0.21777096390724182), ('escalade', 0.21768777072429657)]


In [61]:
result = model_EZ.wv.most_similar(positive=['politique'], topn=20)
print(result)

[('fermete', 0.32823795080184937), ('assimilation', 0.28658562898635864), ('promis', 0.2794075310230255), ('redonner', 0.26249587535858154), ('campagnes', 0.2615153193473816), ('social', 0.2405872344970703), ('reconnaitre', 0.23737871646881104), ('rurale', 0.2343822419643402), ('culturelle', 0.2326223999261856), ('petit', 0.2311442643404007), ('perdu', 0.22779421508312225), ('engage', 0.22116976976394653), ('determination', 0.21908220648765564), ('opposition', 0.2173478603363037), ('moque', 0.21547815203666687), ('origine', 0.2141491025686264), ('reelle', 0.2140859216451645), ('genie', 0.21110787987709045), ('ideologie', 0.20858852565288544), ('mis', 0.20845943689346313)]


In [62]:
result = model_EZ.wv.most_similar(positive=['droit'], topn=20)
print(result)

[('creant', 0.3588884174823761), ('asile', 0.35293877124786377), ('nouvelles', 0.334869384765625), ('genre', 0.32012492418289185), ('domicile', 0.29755154252052307), ('supprimerai', 0.2958025634288788), ('memoire', 0.29257920384407043), ('regroupement', 0.2917586863040924), ('pourront', 0.29166358709335327), ('familiale', 0.2793194651603699), ('proximite', 0.27602800726890564), ('droits', 0.2750615179538727), ('augmenter', 0.2735914885997772), ('individus', 0.27276256680488586), ('usage', 0.2702253460884094), ('familiales', 0.2671177089214325), ('declassement', 0.26558101177215576), ('mains', 0.26471206545829773), ('eleves', 0.2615104615688324), ('permettant', 0.2547591030597687)]


In [63]:
result = model_EZ.wv.most_similar(positive=['entreprise'], topn=20)
print(result)

[('salaries', 0.5631824135780334), ('cout', 0.49999114871025085), ('prime', 0.4928174614906311), ('revaloriser', 0.47768059372901917), ('salaires', 0.4738653004169464), ('eur', 0.446014940738678), ('cotisations', 0.43936601281166077), ('meritants', 0.4365854263305664), ('salarie', 0.432606965303421), ('favoriser', 0.4322202801704407), ('supprimant', 0.41692739725112915), ('entreprises', 0.41094958782196045), ('salaire', 0.40577057003974915), ('veuves', 0.40442559123039246), ('recompenser', 0.4037199020385742), ('fiscalite', 0.4012545645236969), ('transmettre', 0.39700549840927124), ('impot', 0.3969806432723999), ('reduire', 0.3960890471935272), ('remboursement', 0.39510291814804077)]


In [64]:
result = model_EZ.wv.most_similar(positive=['economie'], topn=20)
print(result)

[('favoriser', 0.45505887269973755), ('filiere', 0.44634750485420227), ('augmentant', 0.4456304907798767), ('moyens', 0.42190855741500854), ('concurrence', 0.408406525850296), ('peines', 0.39187076687812805), ('professionnels', 0.3907463848590851), ('entreprises', 0.38777855038642883), ('protegerai', 0.38659316301345825), ('restaurer', 0.37311115860939026), ('transmission', 0.37136110663414), ('revaloriser', 0.36786115169525146), ('ecologie', 0.36750394105911255), ('agriculture', 0.3664856553077698), ('csg', 0.36491474509239197), ('permettre', 0.3642272651195526), ('peche', 0.3623143136501312), ('matiere', 0.36072370409965515), ('production', 0.35707011818885803), ('familiale', 0.3550993502140045)]


In [65]:
result = model_EZ.wv.most_similar(positive=['emploi'], topn=20)
print(result)

[('favoriser', 0.5992624163627625), ('cotisations', 0.5599883198738098), ('supprimant', 0.5059143900871277), ('formation', 0.48450037837028503), ('petites', 0.47176194190979004), ('normes', 0.4707648754119873), ('familiale', 0.4641876816749573), ('augmentant', 0.4556919038295746), ('acces', 0.45011183619499207), ('salaries', 0.4379770755767822), ('supprimerai', 0.43620553612709045), ('moyennes', 0.4327385723590851), ('regime', 0.42698413133621216), ('salaires', 0.419900119304657), ('industrielles', 0.4182604253292084), ('engager', 0.417557030916214), ('fiscalite', 0.4173216223716736), ('revaloriser', 0.4147570729255676), ('investissement', 0.4119020104408264), ('impot', 0.41008663177490234)]


In [66]:
result = model_EZ.wv.most_similar(positive=['travail'], topn=20)
print(result)

[('recompenser', 0.32244688272476196), ('promesses', 0.3100176453590393), ('meritants', 0.2841362953186035), ('patrons', 0.27219098806381226), ('catastrophe', 0.25974249839782715), ('participation', 0.2590811848640442), ('assistanat', 0.25856634974479675), ('bras', 0.2370084822177887), ('moyennes', 0.2367388904094696), ('appel', 0.23248350620269775), ('zero', 0.23199668526649475), ('medecins', 0.22918900847434998), ('salarie', 0.22702588140964508), ('supprimant', 0.22684098780155182), ('resultat', 0.21855764091014862), ('metropoles', 0.20835043489933014), ('revaloriser', 0.2080642431974411), ('meilleurs', 0.2079867571592331), ('prime', 0.2077701985836029), ('souffre', 0.20776771008968353)]


In [67]:
result = model_EZ.wv.most_similar(positive=['taxes'], topn=20)
print(result)

[('impots', 0.6380776762962341), ('baisserai', 0.5964558124542236), ('bureaucratie', 0.5432668924331665), ('taxe', 0.4788958728313446), ('financer', 0.4782668650150299), ('charges', 0.47653722763061523), ('csg', 0.47517073154449463), ('pesent', 0.4708264470100403), ('finance', 0.46961894631385803), ('paye', 0.4579087495803833), ('fiscalite', 0.451343297958374), ('rsa', 0.4506113529205322), ('travailleurs', 0.44486308097839355), ('coute', 0.43048200011253357), ('augmenter', 0.42274633049964905), ('assistanat', 0.42141884565353394), ('baisser', 0.4062364399433136), ('salarie', 0.39495691657066345), ('travaillent', 0.39482608437538147), ('modestes', 0.39450210332870483)]


In [68]:
result = model_EZ.wv.most_similar(positive=['numerique'], topn=20)
print(result)

[('cryptomonnaies', 0.42098867893218994), ('domaine', 0.3874037563800812), ('enquete', 0.3445095717906952), ('strategie', 0.33620819449424744), ('largement', 0.30381959676742554), ('communication', 0.29962730407714844), ('marccormier', 0.29310715198516846), ('equipe', 0.27548903226852417), ('servir', 0.26768574118614197), ('tendances', 0.26234954595565796), ('buzz', 0.26135891675949097), ('international', 0.2588886618614197), ('discuter', 0.25764963030815125), ('revele', 0.2554512321949005), ('zemmoureuse', 0.25529539585113525), ('secteur', 0.25192421674728394), ('tweeter', 0.25037911534309387), ('enjeu', 0.24800635874271393), ('investir', 0.24771273136138916), ('permettent', 0.2474661022424698)]


In [69]:
result = model_EZ.wv.most_similar(positive=['monde'], topn=20)
print(result)

[('impossible', 0.274292916059494), ('dernieres', 0.27307820320129395), ('entier', 0.27277499437332153), ('oui', 0.24932537972927094), ('croient', 0.23521243035793304), ('certainement', 0.22864307463169098), ('pretend', 0.22523535788059235), ('prets', 0.22417058050632477), ('choses', 0.21834832429885864), ('calme', 0.20981739461421967), ('existe', 0.20792962610721588), ('vent', 0.20648017525672913), ('victoire', 0.20347145199775696), ('voit', 0.20307140052318573), ('vecu', 0.20265501737594604), ('enthousiasme', 0.19999386370182037), ('prouve', 0.1963888257741928), ('comprendre', 0.19620592892169952), ('politiques', 0.1901860237121582), ('faux', 0.18768982589244843)]


In [70]:
result = model_EZ.wv.most_similar(positive=['realite'], topn=20)
print(result)

[('ecouter', 0.2802106440067291), ('immense', 0.2754797339439392), ('sujet', 0.27492180466651917), ('agir', 0.2718898355960846), ('choses', 0.2683103084564209), ('penser', 0.2634512782096863), ('existe', 0.2608245313167572), ('subissent', 0.2536970376968384), ('entier', 0.24646571278572083), ('agit', 0.24618542194366455), ('fini', 0.2404419332742691), ('vis', 0.23680269718170166), ('emotion', 0.23479808866977692), ('silence', 0.22882351279258728), ('ultra', 0.2278282195329666), ('autant', 0.22603537142276764), ('professeur', 0.2250305712223053), ('delinquance', 0.22321268916130066), ('bon', 0.21948567032814026), ('elue', 0.21932385861873627)]


In [71]:
result = model_EZ.wv.most_similar(positive=['verite'], topn=20)
print(result)

[('serieusement', 0.3526032865047455), ('profondement', 0.3524889647960663), ('promesses', 0.3514085114002228), ('politicien', 0.344589501619339), ('explique', 0.34267398715019226), ('cet', 0.3383665382862091), ('sincere', 0.3308452367782593), ('plateforme', 0.2977997958660126), ('qui', 0.2936837375164032), ('surpris', 0.29087772965431213), ('considere', 0.27812349796295166), ('raisons', 0.27309224009513855), ('penser', 0.272112101316452), ('parce', 0.27132970094680786), ('pret', 0.27085036039352417), ('voire', 0.26756736636161804), ('democratie', 0.26670438051223755), ('cela', 0.261932373046875), ('perd', 0.2601277828216553), ('fidele', 0.26005151867866516)]


In [72]:
result = model_EZ.wv.most_similar(positive=['societe'], topn=20)
print(result)

[('education', 0.3191291093826294), ('humain', 0.3151134252548218), ('scolaires', 0.31255701184272766), ('corps', 0.3118584156036377), ('fraude', 0.30309951305389404), ('ecole', 0.27593794465065), ('marchandisation', 0.2663460671901703), ('elites', 0.2641723155975342), ('terroristes', 0.26398563385009766), ('ehpad', 0.2637493908405304), ('voie', 0.26191282272338867), ('lutte', 0.2608029544353485), ('symbole', 0.2577558159828186), ('reelle', 0.25535711646080017), ('mener', 0.2502824366092682), ('nation', 0.24934576451778412), ('scandale', 0.24909545481204987), ('enseignement', 0.24652555584907532), ('reconquerir', 0.24354316294193268), ('mondialisation', 0.2427651286125183)]


In [73]:
result = model_EZ.wv.most_similar(positive=['pouvoir'], topn=20)
print(result)

[('armes', 0.3402552306652069), ('augmentant', 0.29585713148117065), ('electeurs', 0.27888283133506775), ('privilegier', 0.2485070526599884), ('parrainages', 0.24770447611808777), ('economies', 0.24505861103534698), ('salaires', 0.2442730814218521), ('davantage', 0.23067881166934967), ('sociale', 0.23020540177822113), ('fiscalite', 0.22395657002925873), ('votre', 0.21960334479808807), ('pesent', 0.21828711032867432), ('droits', 0.2150661200284958), ('ecologie', 0.21413090825080872), ('travers', 0.2131565660238266), ('declassement', 0.20948895812034607), ('bloquer', 0.20941264927387238), ('recompenser', 0.20722264051437378), ('theme', 0.20601971447467804), ('supprimant', 0.2051600217819214)]


In [74]:
result = model_EZ.wv.most_similar(positive=['avenir'], topn=20)
print(result)

[('enjeu', 0.3978812098503113), ('liban', 0.3960678279399872), ('destin', 0.39389678835868835), ('hauteur', 0.3590452969074249), ('commun', 0.3475070297718048), ('jeunesse', 0.33894267678260803), ('ame', 0.3381713032722473), ('cree', 0.33028918504714966), ('transmettre', 0.3247912526130676), ('familles', 0.3099919259548187), ('jeu', 0.3059581518173218), ('esperance', 0.29832732677459717), ('civilisation', 0.2943696677684784), ('mourir', 0.29269152879714966), ('preparer', 0.29171332716941833), ('defendrai', 0.2911350131034851), ('prospere', 0.28013354539871216), ('conserver', 0.27745938301086426), ('parents', 0.27737540006637573), ('meres', 0.27596646547317505)]


In [75]:
result = model_EZ.wv.most_similar(positive=['histoire'], topn=20)
print(result)

[('traditions', 0.40780213475227356), ('ecrire', 0.3223946690559387), ('nation', 0.31167322397232056), ('devenir', 0.30756649374961853), ('soyez', 0.3045843541622162), ('ecrivez', 0.29687440395355225), ('aimer', 0.28828802704811096), ('terroirs', 0.2815338969230652), ('redevenir', 0.28009647130966187), ('conserver', 0.27593860030174255), ('memoire', 0.2697650194168091), ('symbole', 0.2688427269458771), ('serez', 0.2674584984779358), ('liban', 0.2666476368904114), ('langue', 0.26298969984054565), ('prosperite', 0.26144838333129883), ('responsabilite', 0.2585868239402771), ('compatriotes', 0.25243309140205383), ('culture', 0.24593046307563782), ('vivre', 0.24500040709972382)]


In [76]:
result = model_EZ.wv.most_similar(positive=['contre'], topn=20)
print(result)

[('telescopique', 0.3048636317253113), ('matraque', 0.2840363383293152), ('fermer', 0.2780511677265167), ('abat', 0.27615997195243835), ('insecurite', 0.2747453451156616), ('individus', 0.2592676281929016), ('couteau', 0.2582895755767822), ('financiere', 0.2543136477470398), ('ville', 0.24494940042495728), ('denoncer', 0.2425067275762558), ('attentats', 0.23869024217128754), ('stop', 0.2342972457408905), ('sos', 0.23056934773921967), ('fraude', 0.22477716207504272), ('alliancezemmour', 0.22430585324764252), ('parlonsvrai', 0.22136829793453217), ('racisme', 0.22088031470775604), ('humain', 0.2203248292207718), ('islamistes', 0.21984723210334778), ('match', 0.21758848428726196)]


In [77]:
result = model_EZ.wv.most_similar(positive=['faut'], topn=20)
print(result)

[('flux', 0.3146425187587738), ('diplomatique', 0.29348188638687134), ('devons', 0.28987714648246765), ('frontieres', 0.27318382263183594), ('partis', 0.26699793338775635), ('otan', 0.26143738627433777), ('chiffres', 0.25584638118743896), ('lecons', 0.2520400583744049), ('local', 0.250367134809494), ('reprendre', 0.24356594681739807), ('eleves', 0.24160374701023102), ('souhaitent', 0.24017363786697388), ('organiser', 0.22636979818344116), ('permettre', 0.22197821736335754), ('avis', 0.2213335782289505), ('refusent', 0.21767276525497437), ('creant', 0.21339939534664154), ('enjeu', 0.2118528038263321), ('eleve', 0.211468905210495), ('temps', 0.2112819254398346)]


In [78]:
result = model_EZ.wv.most_similar(positive=['crise'], topn=20)
print(result)

[('bloquerai', 0.432441771030426), ('covid', 0.40096890926361084), ('carburant', 0.39510950446128845), ('financiere', 0.38136348128318787), ('ukrainienne', 0.3375910818576813), ('diplomatique', 0.336215615272522), ('plan', 0.33428335189819336), ('actuellement', 0.3249208629131317), ('escalade', 0.32491880655288696), ('manoeuvres', 0.32473400235176086), ('progressiste', 0.32006698846817017), ('gouvernement', 0.31866297125816345), ('euro', 0.31773173809051514), ('bloquer', 0.31493884325027466), ('voie', 0.3036685585975647), ('vente', 0.30218756198883057), ('mise', 0.30076175928115845), ('integrite', 0.2985853850841522), ('ukrainerussie', 0.29817095398902893), ('sanitaire', 0.2976893186569214)]
